In [1]:
###
# The goal of this notebook is to:
# - Take two neural populations
# - Compute the JS divergence between stimuli pairs for each population (the same stimuli pairs)
# - Compute the mutual information between the distributions of JS divergences

import glob
import os
from importlib import reload
import pickle
import datetime

import numpy as np
import scipy as sp
import pandas as pd
import h5py as h5
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import neuraltda.simpComp as sc
import pycuslsa as pyslsa

daystr = datetime.datetime.now().strftime('%Y%m%d')
figsavepth = '/home/brad/DailyLog/'+daystr+'/'
print(figsavepth)

/home/brad/.conda/envs/neuraltda-4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/home/brad/DailyLog/20180606/


In [2]:
# Set up birds and block_paths
birds = ['B1083', 'B1056', 'B1235', 'B1075']
bps = {'B1083': '/home/brad/krista/B1083/P03S03/', 'B1075': '/home/brad/krista/B1075/P01S03/',
       'B1235': '/home/brad/krista/B1235/P02S01/', 'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/',
       'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/'}

# Binning Parameters
windt = 10.0                      # milliseconds
dtovr = 0.5*windt                 # milliseconds
segment_info = [0, 0]             # use full Trial
cluster_group = ['Good']          # use just good clusters
comment = 'JS_MI_TEST'            # BootStrap Populations
bdfs = {}                         # Dictionary to store bdf

In [3]:
# Loop through each bird in our list and bin the data
for bird in birds:
    block_path = bps[bird]
    bfdict = tp2.dag_bin(block_path, windt, segment_info, cluster_group=cluster_group, dt_overlap=dtovr, comment=comment)
    bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
    print(bdf)
    bdfs[bird] = bdf

/home/brad/krista/B1083/P03S03/binned_data/win-10.0_dtovr-5.0_seg-0-0-JS_MI_TEST/20180525T180824Z-10.0-5.0.binned
/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/binned_data/win-10.0_dtovr-5.0_seg-0-0-JS_MI_TEST/20180525T180834Z-10.0-5.0.binned
/home/brad/krista/B1235/P02S01/binned_data/win-10.0_dtovr-5.0_seg-0-0-JS_MI_TEST/20180525T180847Z-10.0-5.0.binned
/home/brad/krista/B1075/P01S03/binned_data/win-10.0_dtovr-5.0_seg-0-0-JS_MI_TEST/20180525T180856Z-10.0-5.0.binned


In [4]:
test_birds = ['B1075', 'B1083','B1056', 'B1235']
test_birds = ['B1056', 'B1235']

In [5]:
# extract population tensors for the populations of interest
population_tensors = {}

for bird in test_birds:
    bdf = bdfs[bird]
    population_tensors[bird] = []
    # open the binned data file
    with h5.File(bdf, 'r') as f:
        stimuli = f.keys()
        print(list(stimuli))
        for stim in sorted(stimuli):
            poptens = np.array(f[stim]['pop_tens'])
            population_tensors[bird].append([poptens, stim])

['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung', 'E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung', 'I_scaled_burung', 'J_scaled_burung', 'K_scaled_burung', 'L_scaled_burung', 'M_scaled_burung', 'N_scaled_burung', 'O_scaled_burung', 'P_scaled_burung']
['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung', 'E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung', 'I_scaled_burung', 'J_scaled_burung', 'K_scaled_burung', 'L_scaled_burung', 'M_scaled_burung', 'N_scaled_burung', 'O_scaled_burung', 'P_scaled_burung']


In [6]:
# flatten the list of population tensors for each population
threshold = 6

def threshold_poptens(tens, thresh):
    ncell, nwins, ntrials = tens.shape
    frs = np.mean(tens, axis=1)
    tfr = thresh*frs
    tfrtens = np.tile(tfr[:, np.newaxis, :], (1, nwins, 1))
    bintens = 1*np.greater(tens, tfrtens)
    return bintens

def shuffle_binmat(binmat):
    ncells, nwin = binmat.shape
    for i in range(ncells):
        binmat[i, :] = np.random.permutation(binmat[i, :])
    return binmat

In [ ]:
popA_binmats = []
popB_binmats = []
popA_SCGs = []
popB_SCGs = []
binmats = [popA_binmats, popB_binmats]
binmats = [[], [], [], []]
SCGs = [[], [], [], []]
for SCG, binmatlist, bird in zip(SCGs, binmats, test_birds):
    bird_tensors = population_tensors[bird]
    for bird_tensor, stim in bird_tensors:
        ncells, nwin, ntrials = bird_tensor.shape
        bin_tensor = threshold_poptens(bird_tensor, threshold)
        for trial in tqdm(range(ntrials)):
            binmatlist.append(bin_tensor[:, :, trial])
            ms = sc.binarytomaxsimplex(bin_tensor[:, :, trial], rDup=True)
            SCG.append(pyslsa.build_SCG(ms))

In [7]:
SCGs = []
for bird in test_birds:
    SCG = []
    binmatlist = []
    bird_tensors = population_tensors[bird]
    for bird_tensor, stim in bird_tensors:
        ncells, nwin, ntrials = bird_tensor.shape
        bin_tensor = threshold_poptens(bird_tensor, threshold)
        for trial in tqdm(range(ntrials)):
            binmatlist.append(bin_tensor[:, :, trial])
            ms = sc.binarytomaxsimplex(bin_tensor[:, :, trial], rDup=True)
            SCG.append(pyslsa.build_SCG(ms))
    SCGs.append(SCG)

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

In [ ]:
# Compute Shuffled complexes
popA_binmats = []
popB_binmats = []
popA_SCGs = []
popB_SCGs = []
binmats = [popA_binmats, popB_binmats]
SCGs = [popA_SCGs, popB_SCGs]
for SCG, binmatlist, bird in zip(SCGs, binmats, test_birds):
    bird_tensors = population_tensors[bird]
    for bird_tensor, stim in bird_tensors:
        ncells, nwin, ntrials = bird_tensor.shape
        bin_tensor = threshold_poptens(bird_tensor, threshold)
        for trial in tqdm(range(ntrials)):
            mat = shuffle_binmat(bin_tensor[:, :, trial])
            binmatlist.append(mat)
            ms = sc.binarytomaxsimplex(mat, rDup=True)
            SCG.append(pyslsa.build_SCG(ms))

In [8]:
# Compute JS popA:

dim = 1
beta = -1.0


for pop, pop_bird in zip(SCGs, test_birds):
    N = len(pop)
    jsmat = np.zeros((N,N))
    for i in tqdm(range(N)):
        for j in tqdm(range(i, N)):
            jsmat[i, j] = pyslsa.cuJS(pop[i], pop[j], dim, beta)
    with open('/home/brad/tmpdata/JSpop_batched{}.pkl'.format(pop_bird), 'wb') as f:
        pickle.dump(jsmat, f)



HBox(children=(IntProgress(value=0, max=320), HTML(value='')))

HBox(children=(IntProgress(value=0, max=320), HTML(value='')))

KeyboardInterrupt: 

In [ ]:
import pickle
with open('/home/brad/tmpdata/JSpopA_dim3.pkl', 'wb') as f:
    pickle.dump(jsmat, f)

In [ ]:
# Compute JS popA:
from joblib import Parallel, delayed
dim = 1
beta = -1.0

for pop in SCGs:
    N = len(pop)
    jsmat = np.zeros((N,N))
    for i in tqdm(range(N)):
        row = Parallel(n_jobs=4)(delayed(pyslsa.cuJS)(pop[i], pop[j], dim, beta) for j in range(i, N))
        jsmat[i, :] = row

In [ ]:
test = np.array([[1,0,0,1], [0,1,1,1]])
test

In [ ]:
shuffle_binmat(test)

In [ ]:
def f(x):
    x = 2*x
    return x

In [ ]:
a = 1
f(a)

In [19]:
len(SCG)

320

In [21]:
test_scgs = SCG[0:320]
%time testout = pyslsa.get_laplacian_spectra(test_scgs, 1)

CPU times: user 7.79 s, sys: 1.8 s, total: 9.59 s
Wall time: 9.54 s


In [24]:
len(testout[319])

126

In [25]:
testout[319]

[-4.46140821765275e-15,
 -3.814545887390271e-15,
 -3.011626356674011e-15,
 -2.715911689151649e-15,
 -1.6447599215413217e-15,
 -1.3124296400694235e-15,
 -1.2699818858548158e-15,
 -1.0339012011294923e-15,
 1.1001308313922274e-16,
 3.1432894822405905e-16,
 1.1153545473500585e-15,
 1.2141704274000834e-15,
 1.6938840887119158e-15,
 2.009334131781525e-15,
 3.706626777547187e-15,
 0.20560590649770952,
 0.29696929332249394,
 0.3718858823169278,
 0.42476521989208244,
 0.5625875557621968,
 0.6957966928801866,
 0.7397068042137829,
 0.8336533067926489,
 0.8935820979944689,
 0.9590659320116305,
 0.9818948687341107,
 1.0121303565311157,
 1.0297998552732188,
 1.249946289045903,
 1.3594462832362415,
 1.3780639254850577,
 1.4754604174384287,
 1.4788869733614736,
 1.6101162410233956,
 1.6699183085765792,
 1.698972408982047,
 1.8324659098131288,
 1.8675699531228593,
 1.9127593157633118,
 1.9662358269257938,
 1.9795386325411155,
 2.023060459016452,
 2.409721086464046,
 2.482848024074114,
 2.57483736908835